# Идея проекта, выборка и предобработка данных
<hr>

> TODO: ссылки на методологию анализа фейков по гистограммам, распознавание лиц в OpenCV, исходный датасет

# Обработка данных
<hr>

> TODO: объем и баланс, хар-ки данных на каждом этапе

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("../dfdc_dataframes/df_total_0_4_compact.json")

In [4]:
# Балансировка классов 1:1

df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])
print(df.shape)
df.head()

(990, 479)


,index,filename,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_8_hist_noise_3,pt_9_hist_noise_3,pt_10_hist_noise_3,pt_11_hist_noise_3,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3
1786,990,pkqrvokzaq.mp4,True,12.982978,26.111126,22.441457,11.663235,12.978255,18.331002,12.118513,...,15.123457,6.481481,1.543210,0.308642,0.925926,1.851852,1.234568,7.407407,4.629630,0.346260
1120,324,blvpaewknn.mp4,True,20.187186,39.080147,33.851511,27.416620,22.192014,33.018917,38.523484,...,21.428571,30.612245,34.183673,47.959184,43.367347,40.306122,44.897959,22.959184,41.326531,0.587204
1655,859,cgnhytdbnt.mp4,True,11.173848,9.010021,11.471094,12.058763,13.885887,14.498021,9.346943,...,4.687500,4.861111,1.215278,10.069444,10.763889,8.159722,6.076389,8.680556,7.986111,0.092111
1720,924,mbikhzndpo.mp4,True,33.920237,24.897523,20.484205,24.209130,22.497744,37.670337,29.216644,...,7.840237,4.881657,4.881657,9.467456,17.307692,17.159763,15.532544,7.100592,14.497041,0.160816
2525,517,trrodhsxof.mp4,True,16.802974,15.391150,10.116138,11.753403,11.220406,16.398418,16.066801,...,4.500000,4.000000,15.000000,18.750000,18.750000,19.000000,17.250000,14.250000,12.750000,0.191137


In [5]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_var,overall_face_hist_std,overall_face_hist_var,overall_face_hist_noise_0.5,overall_face_hist_noise_1,overall_face_hist_noise_3
fake,,,,,,,
True,45.727878,2285.907354,563.795912,462699.545806,0.159494,0.225761,0.257825


In [6]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_var,overall_face_hist_std,overall_face_hist_var,overall_face_hist_noise_0.5,overall_face_hist_noise_1,overall_face_hist_noise_3
fake,,,,,,,
False,49.182392,2608.860611,534.771606,433193.597027,0.200743,0.275005,0.308146


In [7]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_38_hist_var,24.851770,40.804190,0.242970,15.952420
pt_37_hist_var,21.588196,34.900276,0.235660,13.312079
pt_40_hist_var,21.670425,34.904774,0.233925,13.234349
pt_41_hist_var,20.605147,32.470967,0.223562,11.865820
pt_37_var,886.436591,568.411510,0.218597,318.025081
pt_36_hist_var,26.988307,41.971709,0.217277,14.983402
pt_52_var,672.672529,433.669759,0.216030,239.002769
pt_58_hist_var,37.547698,57.545209,0.210294,19.997512
pt_38_var,703.751271,464.338512,0.204961,239.412759
pt_47_hist_var,24.118989,36.438281,0.203432,12.319292


In [8]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
overall_face_hist_noise_0.5,0.200743,0.159494,0.360237
overall_face_hist_noise_1,0.275005,0.225761,0.500766
overall_face_hist_noise_3,0.308146,0.257825,0.565972
pt_7_hist_noise_0.5,1.449724,1.452689,2.902413
pt_8_hist_noise_0.5,1.452484,1.515076,2.967561
pt_9_hist_noise_0.5,1.483363,1.562192,3.045555
pt_10_hist_noise_0.5,1.474690,1.844338,3.319028
pt_6_hist_noise_0.5,1.742977,1.815396,3.558373
pt_24_hist_noise_0.5,2.536658,2.175646,4.712304
pt_56_hist_noise_0.5,2.480267,2.498537,4.978805


In [9]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_38_hist_var,15.952420,65.655959,0.242970
pt_37_hist_var,13.312079,56.488472,0.235660
pt_40_hist_var,13.234349,56.575199,0.233925
pt_41_hist_var,11.865820,53.076114,0.223562
pt_37_var,318.025081,1454.848102,0.218597
pt_36_hist_var,14.983402,68.960016,0.217277
pt_52_var,239.002769,1106.342288,0.216030
pt_58_hist_var,19.997512,95.092907,0.210294
pt_38_var,239.412759,1168.089782,0.204961
pt_47_hist_var,12.319292,60.557269,0.203432


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [10]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(5).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(5).index) +
    list(df_smart.filter(like='std', axis=0).head(3).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

features

['overall_face_hist_noise_1',
 'pt_37_hist_std',
 'pt_58_hist_std',
 'pt_57_hist_std',
 'overall_face_hist_var',
 'pt_6_hist_noise_3',
 'pt_8_hist_noise_3',
 'pt_24_hist_noise_0.5',
 'pt_7_hist_noise_3',
 'pt_57_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_10_hist_noise_0.5',
 'pt_22_hist_noise_0.5',
 'pt_38_hist_noise_3',
 'overall_face_hist_std']

## Разбивка данных на train и test сплиты

In [11]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,pt_55_std,...,pt_8_hist_noise_3,pt_9_hist_noise_3,pt_10_hist_noise_3,pt_11_hist_noise_3,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3
1786,990,True,12.982978,26.111126,22.441457,11.663235,12.978255,18.331002,12.118513,18.739030,...,15.123457,6.481481,1.543210,0.308642,0.925926,1.851852,1.234568,7.407407,4.629630,0.346260
1120,324,True,20.187186,39.080147,33.851511,27.416620,22.192014,33.018917,38.523484,39.951747,...,21.428571,30.612245,34.183673,47.959184,43.367347,40.306122,44.897959,22.959184,41.326531,0.587204
1655,859,True,11.173848,9.010021,11.471094,12.058763,13.885887,14.498021,9.346943,10.827506,...,4.687500,4.861111,1.215278,10.069444,10.763889,8.159722,6.076389,8.680556,7.986111,0.092111
1720,924,True,33.920237,24.897523,20.484205,24.209130,22.497744,37.670337,29.216644,34.632083,...,7.840237,4.881657,4.881657,9.467456,17.307692,17.159763,15.532544,7.100592,14.497041,0.160816
2525,517,True,16.802974,15.391150,10.116138,11.753403,11.220406,16.398418,16.066801,12.145862,...,4.500000,4.000000,15.000000,18.750000,18.750000,19.000000,17.250000,14.250000,12.750000,0.191137


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [102]:
y = df_compact['fake']
X = df_compact.drop(columns=['fake'])
X = SelectKBest(f_classif, k=7).fit_transform(X, y)

In [103]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=42)

In [104]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6
0,258.0,34.412580,22.659363,1184.225694,513.446711,1252.512539,15.625000
1,184.0,34.208323,38.653281,1170.209375,1494.076100,2103.944244,29.000000
2,291.0,27.808537,27.055475,773.314733,731.998715,579.906508,22.520661
3,4.0,26.892045,30.014151,723.182073,900.849277,793.699923,17.307692
4,1220.0,50.693916,43.093966,2569.873103,1857.089867,2337.070630,21.745562


In [105]:
y_train.head()

2266    False
671      True
754      True
800      True
1983    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [106]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [107]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [108]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### Плохие классфикаторы

- LogisticRegression() 
- SGDClassifier() 
- SVC(gamma='auto') 

### NN - MLPClassifier (f1=0.59) [to be modified]

In [109]:
print_polynomial_pipeline(
    # MLPClassifier(), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(5, 7, 7, 3), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=150000,
        max_iter=50000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[73 32]
 [43 50]]
               precision    recall  f1-score   support

 class "real"       0.63      0.70      0.66       105
class "fakes"       0.61      0.54      0.57        93

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



### RandomForestClassifier (good, f1=60..65) [to be modified] 

In [155]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test
) 

[[51 54]
 [27 66]]
               precision    recall  f1-score   support

 class "real"       0.65      0.49      0.56       105
class "fakes"       0.55      0.71      0.62        93

     accuracy                           0.59       198
    macro avg       0.60      0.60      0.59       198
 weighted avg       0.61      0.59      0.59       198



### DecisionTreeClassifier (f1=55..60) [to be modified]

In [126]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=3, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[51 54]
 [35 58]]
               precision    recall  f1-score   support

 class "real"       0.59      0.49      0.53       105
class "fakes"       0.52      0.62      0.57        93

     accuracy                           0.55       198
    macro avg       0.56      0.55      0.55       198
 weighted avg       0.56      0.55      0.55       198



### RadiusNeighborsClassifier (f1=55..59) [уязвим к выбросам]

In [144]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=6.5, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[34 71]
 [14 79]]
               precision    recall  f1-score   support

 class "real"       0.71      0.32      0.44       105
class "fakes"       0.53      0.85      0.65        93

     accuracy                           0.57       198
    macro avg       0.62      0.59      0.55       198
 weighted avg       0.62      0.57      0.54       198



### KNeighborsClassifier (f1=0.58) [to be modified]

In [150]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=3, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)

[[54 51]
 [25 68]]
               precision    recall  f1-score   support

 class "real"       0.68      0.51      0.59       105
class "fakes"       0.57      0.73      0.64        93

     accuracy                           0.62       198
    macro avg       0.63      0.62      0.61       198
 weighted avg       0.63      0.62      0.61       198

